In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.feature_selection import RFE
import joblib

In [2]:
#modifying dataset to make it numerical
df=pd.DataFrame(pd.read_csv(r'C:\Users\nihca\OneDrive\Documents\vscode\edaproject\dataset.csv'))
df=df[df["Disease"].notna()]
all_symptoms = set()

symptom_cols = [col for col in df.columns if col.startswith("Symptom_")]


for col in symptom_cols:
    symptoms = df[col].dropna().unique()
    all_symptoms.update(symptoms)

new_df = pd.DataFrame()
new_df["Disease"] = df["Disease"]  


for symptom in all_symptoms:
    if symptom and str(symptom).strip(): 
        new_df[str(symptom)] = 0 
        
        for col in symptom_cols:
            mask = df[col] == symptom
            original_indices = df.loc[mask].index
            new_df.loc[original_indices, str(symptom)] = 1


df = new_df
features=df.drop(columns=["Disease"])
df

C:\Users\nihca\AppData\Local\Temp\ipykernel_2564\2033040679.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[str(symptom)] = 0
C:\Users\nihca\AppData\Local\Temp\ipykernel_2564\2033040679.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[str(symptom)] = 0
C:\Users\nihca\AppData\Local\Temp\ipykernel_2564\2033040679.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

,Disease,redness_of_eyes,skin_peeling,coma,throat_irritation,blood_in_sputum,swollen_legs,receiving_blood_transfusion,muscle_weakness,restlessness,...,diarrhoea,irregular_sugar_level,continuous_sneezing,swollen_extremeties,vomiting,joint_pain,dischromic _patches,swollen_blood_vessels,polyuria,swelled_lymph_nodes
0,Fungal infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,Fungal infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,Fungal infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,Fungal infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Fungal infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4916,Acne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4917,Urinary tract infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4918,Psoriasis,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
#list of all unique diseases
df['Disease'].unique()

array(['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes ',
       'Gastroenteritis', 'Bronchial Asthma', 'Hypertension ', 'Migraine',
       'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice',
       'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

In [4]:
#assigning numerical value for disease
le=LabelEncoder()
df['Disease']=le.fit_transform(df['Disease'])
X=df.drop(columns=["Disease"],axis=1)
y=df["Disease"]

In [5]:
#splitting training and testing data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)



In [6]:
#applying standard scaler to standardize the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.columns),
    ]
)
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [7]:
#model definition
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [8]:
#applying a sample test to get accuracy
y_pred = logreg.predict(X_test)
y_pred_prob = logreg.predict_proba(X_test)
accuracy = accuracy_score(y_test, y_pred)


In [9]:
#joblib.dump(logreg, 'logistic_model.pkl')
#joblib.dump(preprocessor, 'preprocessor.pkl')
#joblib.dump(le, 'label_encoder.pkl')